In [1]:
import jax
jax.config.update('jax_platform_name', 'cpu')

In [2]:
from compute import Var, Par
from datastructures.api import Model, adda, addf, addsolver, setsolvefor
from datastructures.execution import edges_from_components
from datastructures.graphutils import namefromid, dfs_tree, SOLVER, VAR, COMP, nested_sources, sources, edges_to_Ein_Eout, flat_graph_formulation, root_solver
from datastructures.nestedgraph import root_sources
from datastructures.execution import generate_components_and_residuals
from datastructures.workflow import get_f, order_from_tree, default_solver_options, mdao_workflow, implicit_comp_name
from datastructures.workflow_mdao import mdao_workflow_with_args
from datastructures.operators import sort_scc, reorder_merge_solve
from datastructures.assembly import build_archi
import openmdao.api as om
from datastructures.api import evalexpr

In [3]:
from sympy import cos

In [4]:
model = Model()
m = model.root

In [5]:
D_f = Var('D_f', unit='m')

In [7]:
A_s = adda(m, 'A_s', 1.5*(D_f/2)**2, 'm**2')

In [8]:
eta_s = Par(r'\eta_s', 0.27)
phi_s = Par(r'\phi_s', 800, 'W/m**2')
theta_bar = Par(r'\theta', 55, 'deg')
I_d = Par('I_d', 0.9)
D = Par('D', 0.005)
L_s = Par('L_s', 10)

In [10]:
adda(m, 'P_recharge', 
    eta_s * phi_s * cos(theta_bar) * I_d * (1-D)**L_s * A_s, unit='W', forceunit=True)

P_recharge

# Parameters

## Mahas equations

In [ ]:
model = Model()
m = model.root
D_d = Var('D_d', 5, 'm')
D_f = Var('D_f', 5, 'm')
D_s = Var('D_s', 1, 'm')
t_d = Var('t_d', 0.5, 'm')
t_f = Var('t_f', 2, 'm')
t_s = Var('t_s', 6, 'm')

P_hotel = Par('P_hotel', 50, 'W')
eta_s = Par('eta_s', 0.27)
phi_s = Par('phi_s', 800, 'W/m**2')
theta_bar = Par('theta_bar', 55, 'deg')
I_d = Par('I_d', 0.9)
D = Par('D', 0.005)
L_s = Par('L_s', 10)
t_r = Var('t_r', unit='hr')
E_AUV = Par('E_AUV', 1.9, 'kW*hr')
t_service = Par('t_service', 12, 'hr')
t_comms = Par('t_comms', 1, 'hr')
gamma = Par('gamma', 2)
A_s = adda(m, 'A_s', 1.5*(D_f/2)**2, 'm**2')
t_mission = Par('t_mission', 24, 'hr')

# AUV recharging system
P_recharge = adda(m, 'P_recharge', 
    eta_s * phi_s * sp.cos(theta_bar) * I_d * (1-D)**L_s * A_s, unit='W')
P_drawrecharge = adda(m, 'P_drawrecharge', P_hotel - P_recharge, unit='W')
E_recharge_gen = adda(m, 'E_recharge_gen', P_recharge * t_r, unit='W*hr')
E_service = adda(m, 'E_service', E_AUV * gamma, unit='W*hr')
P_service = adda(m, 'P_service', E_service / t_service, unit='W')
P_drawservice = adda(m, 'P_drawservice', P_hotel - P_service, unit='W')

# Propulsion
rho = Par('rho', 1023.6, 'kg/m**3') # density of seawater [kg/m^3]
C_d = Par('C_d', 1) # estimate drag coefficient (a square flat plate at 90 deg to the flow is 1.17)
S_w = Par('S_w', 0.5, 'm**2') # will technically need to determine from A_solar and other structural needs... assume something for now
v = Var('v', 0.3, 'm/s') # [m/s]
eta_m = Par('eta_m', 0.75) # estimated, need to determine from motors?
P_move = adda(m, 'P_move', rho*C_d*S_w*v**3/(2*eta_m), unit='W')

# Comms beginning
P_comms = Var('P_c', unit='W')

# Balances
E_required = adda(m, 'E_required', E_service + 
    (P_hotel + P_move)*t_mission + P_comms*t_comms, unit='W*hr')

R1 = addf(m, E_recharge_gen-E_required, name='R_1')

# Battery
mu_batt = Par('mu_batt', 30, 'W*hr/kg')
DOD = Par('DOD', 0.4)
eta_trans = Par('eta_trans', 0.85)
nu_batt = Par('nu_batt', 450, 'kW*hr/(m**3)')
N = Par('N', 1)
C = adda(m, 'C', E_service/(DOD*N*eta_trans), unit='kW*hr')
m_batt_zero = Par('m_batt_zero', 5, 'kg')
m_batt = adda(m, 'm_batt', C/mu_batt + m_batt_zero, unit='kg')
V_batt = adda(m, 'V_batt', C/nu_batt, unit='m**3')

# Comms
k = Par('k', 1.38065e-23, 'J/K')
c = Par('c', 3e8, 'm/s')
f = Par('f', 2.2, 'GHz')
lambd = adda(m, '\\lambda', c/f, unit='m')
eta_parab = Par('eta_parab', 0.55)
theta_t = Par('theta_t', 32)
error_t = Par('error_t', 27)
G_pt_db = adda(m, 'G_pt', 44.3-dec2db(theta_t**2))
L_pt_db = adda(m, 'L_pt_db', -12*(error_t/theta_t)**2)
G_t = adda(m, 'G_t', db2dec(G_pt_db+L_pt_db))
D_r = Par('D_r', 5.3, 'm')
G_rp = adda(m, 'G_rp', eta_parab*(np.pi*D_r/lambd)**2)
L_pr = Par('L_pr', db2dec(-0.1))
G_r = adda(m, 'G_r', G_rp*L_pr)
BER = Par('BER', 0.0001)
R = Var('R', unit='1/micros')
h = Par('h', 500, 'km')
Re = Par('Re', 6378, 'km')
T_s = Par('T_s', 135, 'K')
S = adda(m, 'S', sp.sqrt(h*(h+2*Re)), unit='km')
L_a = Par('L_a', db2dec(-0.3))
L_sp = adda(m, 'L_sp', (lambd/(4*np.pi*S))**2)
L_l = Par('L_l', db2dec(-1))
EN = adda(m, 'EN', P_comms*L_a*L_sp*L_l*L_pr*G_r*G_t/(k*T_s*R))
EN_req = Par('EN_{req}', 30)

R2 = addf(m, EN-EN_req, name='R_2')